In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import datetime
import os
import sys
from get_unique_resources_data import get_unique_resources_data
from get_printable_resource_names import get_printable_resource_names
import matplotlib.pyplot as plt

update later to include utils

In [2]:
# read in a file starting from the root directory in data/a_upd_generator_df.csv
generator_df = pd.read_csv('../data/a_upd_generator_df.csv')


In [3]:
case_names = [    
              "Thermal_Base",
              "2_Hr_BESS", 
              "2_Hr_BESS_Fuelx2",
              "4_Hr_BESS",
              "4_Hr_BESS_Fuelx2",
              "4_Hr_BESS_Fuelx3",
              "4_Hr_BESS_Fuelx4",
              "6_Hr_BESS",
              "6_Hr_BESS_Fuelx2",
              "8_Hr_BESS",
              "8_Hr_BESS_Fuelx2",
              "10_Hr_BESS",
              "10_Hr_BESS_Fuelx2",
              ]

current_dir = os.getcwd()
print(current_dir)

plots_path = os.path.join(current_dir, 'plots') + "/"
latex_path = os.path.join(current_dir, 'latex') + "/"
csv_path = os.path.join(current_dir, 'csv') + "/"
if not os.path.exists(plots_path):
    os.makedirs(plots_path)
if not os.path.exists(csv_path):
    os.makedirs(csv_path)
if not os.path.exists(latex_path):
    os.makedirs(latex_path)


cem_path = os.path.join(os.path.dirname(current_dir), 'GenX.jl', 'research_systems')
policies_path = os.path.join(os.path.dirname(current_dir), 'SPCM', 'research_systems')

date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

C:\Users\ks885\Documents\aa_research\Modeling\spcm_genx_experiment\figures


In [4]:
# get unique generator names
unique_resources, cases_resources_capacities = get_unique_resources_data(case_names, policies_path)


adding resource: NG 2-on-1 Combined Cycle (F-Frame) from case: Thermal_Base
adding resource: NG Combustion Turbine (F-Frame) from case: Thermal_Base
adding resource: Land-Based Wind - Class 1 - Technology 1 from case: Thermal_Base
adding resource: Utility PV - Class 1 from case: Thermal_Base
adding resource: Utility-Scale Battery Storage - 2Hr from case: 2_Hr_BESS
adding resource: Utility-Scale Battery Storage - 4Hr from case: 4_Hr_BESS
adding resource: Utility-Scale Battery Storage - 6Hr from case: 6_Hr_BESS
adding resource: Utility-Scale Battery Storage - 8Hr from case: 8_Hr_BESS
adding resource: Utility-Scale Battery Storage - 10Hr from case: 10_Hr_BESS


In [5]:
column_name_map = {
    'Inv_Cost_per_MWyr': 'Investment [$/MW-Yr]',
    'Inv_Cost_per_MWhyr': 'Investment [$/MWh-Yr]',
    'Fixed_OM_Cost_per_MWyr': 'FOM [$/MW-Yr]',
    'Var_OM_Cost_per_MWh': 'VOM [$/MWh]'
}

In [6]:
# get printable resource names
printable_resource_names = get_printable_resource_names(unique_resources)

In [7]:
# create dataframe with unique resources as rows
resources_cost_df = pd.DataFrame(index=unique_resources)


In [8]:
# add columns for each resources based on data in generator_df using the column name map
for col, new_col in column_name_map.items():
    resources_cost_df[new_col] = resources_cost_df.index.map(
        lambda res: generator_df.loc[generator_df['Resource'] == res, col].values[0]
        if res in generator_df['Resource'].values else np.nan
    )

In [9]:
# round all numbers to zero decimal places and make sure they are integers
resources_cost_df = resources_cost_df.round(2)
# make the numers that are not VOM integers
resources_cost_df['Investment [$/MW-Yr]'] = resources_cost_df['Investment [$/MW-Yr]'].astype(int)
resources_cost_df['Investment [$/MWh-Yr]'] = resources_cost_df['Investment [$/MWh-Yr]'].astype(int)
resources_cost_df['FOM [$/MW-Yr]'] = resources_cost_df['FOM [$/MW-Yr]'].astype(int)
resources_cost_df['VOM [$/MWh]'] = resources_cost_df['VOM [$/MWh]'].astype(float)

In [10]:
# print a csv
resources_cost_df.to_csv(csv_path + 'table1_resources_costs.csv')


In [11]:
latex_table = r"""\begin{table}[htp]
\centering
{\begin{tabular}{lcccc}
\hline
\multirow{2}{*}{Resource} & Investment & Investment & FOM & VOM \\
 & {[}$\$/MW$-Yr{]} & {[}$\$/MWh$-Yr{]} & {[}$\$/MW$-Yr{]} & {[}$\$/MWh${]} \\ \hline
"""

for resource, row in resources_cost_df.iterrows():
    inv_mwyr = int(row['Investment [$/MW-Yr]'])
    inv_mwhyr = int(row['Investment [$/MWh-Yr]'])
    fom = int(row['FOM [$/MW-Yr]'])
    vom = row['VOM [$/MWh]']
    if vom == 0 or vom == 0.0:
        vom_str = f"{int(vom)}"
    else:
        vom_str = f"{vom:.2f}"
    latex_table += f"{resource} & {inv_mwyr:,} & {inv_mwhyr:,} & {fom:,} & {vom_str} \\\\\n"

latex_table += r"""\hline
\end{tabular}}
\captionof{table}{Initial Costs for Candidate Technologies in Simulations} \label{tab:res_costs} 
\end{table}
"""

print(latex_table)
with open(latex_path + 'table1_resources_costs.tex', 'w') as f:
    f.write(latex_table)


\begin{table}[htp]
\centering
{\begin{tabular}{lcccc}
\hline
\multirow{2}{*}{Resource} & Investment & Investment & FOM & VOM \\
 & {[}$\$/MW$-Yr{]} & {[}$\$/MWh$-Yr{]} & {[}$\$/MW$-Yr{]} & {[}$\$/MWh${]} \\ \hline
NG 2-on-1 Combined Cycle (F-Frame) & 74,962 & 0 & 34,500 & 2.17 \\
NG Combustion Turbine (F-Frame) & 67,440 & 0 & 26,100 & 6.94 \\
Land-Based Wind - Class 1 - Technology 1 & 83,289 & 0 & 32,443 & 0.10 \\
Utility PV - Class 1 & 74,134 & 0 & 23,765 & 0.10 \\
Utility-Scale Battery Storage - 2Hr & 64,517 & 0 & 28,621 & 0 \\
Utility-Scale Battery Storage - 4Hr & 105,038 & 0 & 48,168 & 0 \\
Utility-Scale Battery Storage - 6Hr & 145,560 & 0 & 67,714 & 0 \\
Utility-Scale Battery Storage - 8Hr & 186,081 & 0 & 87,260 & 0 \\
Utility-Scale Battery Storage - 10Hr & 226,602 & 0 & 106,807 & 0 \\
\hline
\end{tabular}}
\captionof{table}{Initial Costs for Candidate Technologies in Simulations} \label{tab:res_costs} 
\end{table}

